# 要約 
このノートブックは、Kaggleの「LMSYS - Chatbot Arena」コンペティションに関連するデータセットを使用して、自動応答の好みを予測するモデルを構築するためのものです。具体的には、異なる大規模言語モデル（LLM）から生成された応答に基づき、どちらの応答がユーザーに好まれるかを予測する問題に取り組んでいます。

### 問題の概要:
- データセットは、ユーザーからのプロンプトに対する2つの異なる応答（応答Aと応答B）と、どちらの応答が勝者であるか（winner_model_aおよびwinner_model_b）を含みます。ノートブックではこれらのデータを用いて、ユーザーの選好を予測するモデルを開発します。

### 主な手法:
1. **データの読み込みと前処理**:
   - `pandas`を用いてCSVファイルからデータを読み込み、プロンプトと応答をリストとして抽出します。
   
2. **言語検出**:
   - `langdetect`ライブラリを使用して、プロンプトと応答の言語を検出します。この情報をもとに、ユニークな言語のカウントなどを行います。

3. **データセットの準備**:
   - `sklearn`の`train_test_split`を利用して、データを訓練セットと検証セットに分割します。
   - プロンプトと応答をトークナイズするために、`transformers`ライブラリを用いてBERTトークナイザーを利用します。

4. **モデルの訓練**:
   - BERTをベースとした分類モデル（`BertForSequenceClassification`）を初期化し、AdamWオプティマイザーを用いて訓練を行います。
   - クロスエントロピー損失を使用した訓練ループが含まれ、エポックごとに訓練損失と検証精度が表示されます。

5. **結果の保存**:
   - 訓練が完了したモデルは、PyTorchを用いて保存されます。

### 使用ライブラリ:
- `numpy`と`pandas`: 数値計算とデータ操作。
- `langdetect`: テキストの言語を検出。
- `transformers`: BERTモデルのトークナイズと分類タスクの実行。
- `torch`: 深層学習フレームワークとして使用。

このノートブックは、プログラム全体を通じて言語検出からデータの準備、モデルの構築と訓練までを含む、包括的な機械学習ワークフローを示しています。

---


# 用語概説 
以下は、ノートブック内で使用されている専門用語の簡単な解説です。主に初心者には馴染みが薄い用語や、実務経験がないと理解が難しいものに焦点を当てました。

1. **langdetect**:
   - 自然言語処理のライブラリで、テキストから言語を自動的に検出するためのツール。多言語データを扱う際に役立つ。

2. **Counter**:
   - Pythonの`collections`ライブラリに含まれるクラスで、要素の頻度をカウントするのに使用される。特に、データの出現数を把握するために便利。

3. **トークナイザー**:
   - 自然言語処理において、テキストをトークン（単語や文の構成要素）に分割するコンポーネント。BERTなどのモデルでは、入力テキストをトークン化することが重要。

4. **注意マスク (Attention Mask)**:
   - モデルに入力する際、どのトークンが有効でどのトークンが無効かを示すためのマスク。Paddingトークンなどを無視して、意味のあるトークンのみに焦点を当てるために使用される。

5. **TensorDataset**:
   - PyTorchでデータセットを管理するためのデータ構造。テンソルを複数組み合わせて一つのデータセットとして扱うことができる。

6. **DataLoader**:
   - PyTorchのイテレーターで、データセットをバッチ単位で管理し、訓練プロセス中にバッチを簡単に取り出せるようにする。これにより、大規模なデータを安全に処理できる。

7. **AdamWオプティマイザー**:
   - 学習率を調整しながら効率的にパラメータを更新するためのアルゴリズムの一つ。通常のAdamにWeight Decay（重み減衰）を組み合わせたもので、過学習を防ぐ効果がある。

8. **lr_scheduler (学習率スケジューラー)**:
   - 学習率を訓練中に調整するための方法。適切な学習率を維持することで、モデルの性能を向上させることができる。

9. **クロスエントロピー損失関数**:
   - 分類問題において最も一般的に使用される損失関数。モデルの出力確率と実際のラベルとの間の距離を測り、モデルの学習を促進するために使用。

10. **logits**:
    - モデルの出力層の前にある生のスコア。通常、これをソフトマックス関数に通すことで、確率として解釈できるようになる。

これらの用語の理解は、このノートブックのコードを追うために有用であり、初心者が深層学習のフレームワークを使ってモデルを構築する際に役立つでしょう。

---


In [1]:
# このPython 3環境には、多くの役立つ分析ライブラリがインストールされています
# これは、kaggle/python Dockerイメージによって定義されています: https://github.com/kaggle/docker-python
# たとえば、ここではいくつかの役立つパッケージを読み込むことができます

import numpy as np # 線形代数
import pandas as pd # データ処理、CSVファイルの入出力（例: pd.read_csv）

# 入力データファイルは読み取り専用の"../input/"ディレクトリにあります
# たとえば、これを実行すると（クリックするかShift+Enterを押すことによって）、入力ディレクトリの下のすべてのファイルがリストアップされます
print(pd.__version__)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# 現在のディレクトリ（/kaggle/working/）に最大20GBのデータを書き込むことができ、"Save & Run All"を使用してバージョンを作成すると出力として保存されます
# 一時ファイルを/kaggle/temp/に書き込むこともできますが、現在のセッションの外には保存されません

2.2.2
/kaggle/input/lmsys-chatbot-arena/sample_submission.csv
/kaggle/input/lmsys-chatbot-arena/train.csv
/kaggle/input/lmsys-chatbot-arena/test.csv


In [3]:
df = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv') # 'train.csv'ファイルを読み込みます
df.dtypes # 各列のデータ型を表示します

id                 int64
model_a           object
model_b           object
prompt            object
response_a        object
response_b        object
winner_model_a     int64
winner_model_b     int64
winner_tie         int64
dtype: object

In [ ]:
from langdetect import detect
from collections import Counter

# テキスト内の言語を検出する関数
def detect_language(text):
    try:
        return detect(text) # 言語を検出して返す
    except:
        return 'unknown'  # 言語検出に失敗した場合

# プロンプトと応答を抽出
prompts = df['prompt'].tolist() # プロンプトをリストとして取得
responses_a= df['response_a'].tolist() # 応答Aをリストとして取得
responses_b = df['response_b'].to_list() # 応答Bをリストとして取得
tot = responses_a + responses_b # 応答Aと応答Bを結合して一つのリストにする
print(prompts) # プロンプトを表示
print(tot) # 結合された応答を表示
# プロンプトと応答の言語を検出
prompt_languages = [detect_language(prompt) for prompt in prompts] # 各プロンプトの言語を検出
response_languages = [detect_language(tot) for response in responses] # 応答の言語を検出（ただし、此処で' responses 'は未定義のためエラーになります）
# Combine all detected languages
all_languages = prompt_languages + response_languages # 検出されたすべての言語を結合

# ユニークな言語の数をカウント
unique_languages = Counter(all_languages) # ユニークな言語をカウントする

print(f"Number of unique languages detected: {len(unique_languages)}") # 検出されたユニークな言語の数を表示
print("Languages detected:", unique_languages.keys()) # 検出された言語を表示

In [4]:
!pip install langdetect # langdetectライブラリをインストールします

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=97b635e2d8dd7396e1736e2e3e2974aa82aa0b4fb4cf3c6dc5a475f7985d5113
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split

# CSVからデータセットを読み込む
# df = pd.read_csv('your_dataset.csv') # あなたのデータセットを読み込む

# 入力（プロンプトと応答）とラベル（勝者）を分離
prompts = df['prompt'].tolist() # プロンプトをリストとして取得
responses_a = df['response_a'].tolist() # 応答Aをリストとして取得
responses_b = df['response_b'].tolist() # 応答Bをリストとして取得
labels_a = df['winner_model_a'].tolist() # モデルAの勝者ラベルをリストとして取得
labels_b = df['winner_model_b'].tolist() # モデルBの勝者ラベルをリストとして取得

# 応答とラベルをモデルAとモデルBのために組み合わせる
responses = [(resp_a, resp_b) for resp_a, resp_b in zip(responses_a, responses_b)] # 応答をペアにする
labels = [(label_a, label_b) for label_a, label_b in zip(labels_a, labels_b)] # ラベルをペアにする

print("------------------") # 区切りのための印

------------------


In [4]:
!pip install transformers # transformersライブラリをインストールします

In [18]:
import logging

def disable_logging_during_tests():
    # 後で元のログレベルを復元するために現在のログレベルを保存
    original_log_level = logging.getLogger().getEffectiveLevel()

    # ログレベルを高いレベルに設定（例: WARNINGまたはCRITICAL）
    logging.disable(logging.CRITICAL)

    # ここでテストを実行

    # テスト後に元のログレベルを復元
    logging.disable(original_log_level)

# テストを実行する前にこの関数を呼び出す
disable_logging_during_tests()

In [19]:
from transformers import BertTokenizer

# トークナイザーの初期化
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',device_map='auto') # BERTトークナイザーを事前学習モデルから読み込む
tokenizer # トークナイザーの情報を表示

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [20]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # 使用するデバイス（GPUまたはCPU）を設定
print('Using device:', device) # 使用するデバイスを表示

Using device: cuda


In [ ]:
# プロンプトと応答をトークナイズ（文字列をトークン化）
tokenized_inputs = [] # トークン化された入力を格納するリスト
for prompt, (resp_a, resp_b) in zip(prompts, responses): # プロンプトとペアにした応答を取得
    inputs_a = tokenizer(prompt, resp_a, padding='max_length', truncation=True, max_length=512, return_tensors='pt').to(device) # 応答Aをトークン化
    inputs_b = tokenizer(prompt, resp_b, padding='max_length', truncation=True, max_length=512, return_tensors='pt').to(device) # 応答Bをトークン化
#     print(inputs_a) # トークン化された応答Aを表示（必要に応じて使用）
    tokenized_inputs.append((inputs_a, inputs_b)) # トークン化された応答AとBをリストに追加

# 入力テンソルの準備
input_ids = torch.cat([torch.cat([item[0]['input_ids'], item[1]['input_ids']], dim=0) for item in tokenized_inputs], dim=0).to(device) # トークン化された入力IDを準備
attention_masks = torch.cat([torch.cat([item[0]['attention_mask'], item[1]['attention_mask']], dim=0) for item in tokenized_inputs], dim=0).to(device) # 注意マスクを準備
labels_tensor = torch.tensor(labels) # ラベルのテンソルを生成

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

# データを訓練セットと検証セットに分割
train_inputs, val_inputs, train_masks, val_masks, train_labels, val_labels = train_test_split(
    input_ids, attention_masks, labels_tensor, random_state=42, test_size=0.2)

# 訓練用のTensorDatasetとDataLoaderを作成する
train_dataset = TensorDataset(train_inputs, train_masks, train_labels) # 訓練データセットを作成
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True) # 訓練データローダーを作成

# 検証用のTensorDatasetとDataLoaderを作成する
val_dataset = TensorDataset(val_inputs, val_masks, val_labels) # 検証データセットを作成
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False) # 検証データローダーを作成

In [ ]:
from transformers import BertForSequenceClassification, AdamW
import torch.nn as nn
import torch.optim as optim

# モデルの初期化
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2) # BERTモデルの初期化
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # 使用するデバイスを設定
model.to(device) # モデルをデバイスに移動させる

# オプティマイザーとスケジューラー
optimizer = AdamW(model.parameters(), lr=1e-5) # AdamWオプティマイザーを設定
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1) # 学習率スケジューラーの設定

# トレーニングループ
num_epochs = 3 # エポック数
criterion = nn.CrossEntropyLoss() # クロスエントロピー損失関数を設定

for epoch in range(num_epochs):
    model.train() # モデルを訓練モードに切り替え
    total_loss = 0 # 総損失の初期化
    for batch in train_dataloader:
        batch = tuple(t.to(device) for t in batch) # バッチデータをデバイスに転送
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]} # 入力を辞書として準備
        outputs = model(**inputs) # モデルで出力を取得
        loss = outputs.loss # 損失を取得
        total_loss += loss.item() # 総損失に加算
        
        optimizer.zero_grad() # 勾配をゼロに初期化
        loss.backward() # 損失の勾配を逆伝播
        optimizer.step() # パラメータの更新
    
    avg_train_loss = total_loss / len(train_dataloader) # 平均訓練損失を算出
    print(f'Epoch {epoch + 1}/{num_epochs}, Training Loss: {avg_train_loss}') # エポックごとの平均訓練損失を表示

    # 検証
    model.eval() # モデルを評価モードに切り替え
    val_loss = 0 # 検証損失の初期化
    val_correct = 0 # 正解数の初期化
    with torch.no_grad(): # 勾配計算を無効にして評価
        for batch in val_dataloader:
            batch = tuple(t.to(device) for t in batch) # バッチデータをデバイスに転送
            inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]} # 入力を辞書として準備
            outputs = model(**inputs) # モデルで出力を取得
            loss = outputs.loss # 損失を取得
            val_loss += loss.item() # 検証損失に加算
            
            logits = outputs.logits # 出力ロジットを取得
            _, preds = torch.max(logits, dim=1) # 最大のロジットを持つ予測ラベルを取得
            val_correct += torch.sum(preds == batch[2]).item() # 正解数をカウント
    
    avg_val_loss = val_loss / len(val_dataloader) # 平均検証損失を算出
    val_accuracy = val_correct / len(val_dataset) # 検証精度を算出
    print(f'Validation Loss: {avg_val_loss}, Validation Accuracy: {val_accuracy}') # 検証結果を表示

# モデルを保存
torch.save(model.state_dict(), 'bert_model.pth') # 訓練したモデルの状態を保存